# UD1 Randomized Algorithms and Concentration Inequalities  
**Course:** Quantum Machine Learning and Quantumn computers (UIMP 2025-2026 course)

**Name:** Ferrán Mata Infante

In [144]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, uniform, beta

First, we will select the parameters of the distributions to be used. We will study the Gaussian distribution with σ greater and smaller than 1, as well as the uniform and beta distributions

In [145]:
# Gaussian distribution 1
mu_gaussian_1 = 2 #Mean value
sigma_gaussian_1 = 1.5 #Sigma

# Gaussian distribution 2
mu_gaussian_2 = 2 #Mean value
sigma_gaussian_2 = 0.5 #Sigma

# Uniform distribution
a_uniform=0
b_uniform=1

# Beta distribution
alpha_beta, beta_beta= 3,5


Now, we're going to plot the function distributions



In [171]:
x_gaussian = np.linspace(-6,10, 200)
x_uniform = np.linspace(a_uniform, b_uniform, 200)
x_beta = np.linspace(0, 1, 200)

plt.figure(figsize=(18, 6), dpi=1000) 

# --- Gaussian ---
plt.subplot(1, 3, 1)
plt.plot(x_gaussian, norm.pdf(x_gaussian, mu_gaussian_1, sigma_gaussian_1), color='red', lw=2, label=f'Theoretical PDF for N({mu_gaussian_1}, {sigma_gaussian_1})')
plt.plot(x_gaussian, norm.pdf(x_gaussian, mu_gaussian_2, sigma_gaussian_2), color='blue', lw=2, label=f'Theoretical PDF for N({mu_gaussian_2}, {sigma_gaussian_2})')
plt.title("Gaussian Distribution")
plt.xlabel("Value")
plt.ylabel("Density")
plt.legend()

# --- Uniform ---
plt.subplot(1, 3, 2)
plt.plot(x_uniform, uniform.pdf(x_uniform, a_uniform, b_uniform-a_uniform), color='red', lw=2, label='Theoretical PDF')
plt.title("Uniform Distribution")
plt.xlabel("Value")
plt.ylabel("Density")
plt.legend()

# --- Beta ---
plt.subplot(1, 3, 3)
plt.plot(x_beta, beta.pdf(x_beta, alpha_beta, beta_beta), color='red', lw=2, label='Theoretical PDF')
plt.title("Beta Distribution")
plt.xlabel("Value")
plt.ylabel("Density")
plt.legend()

plt.tight_layout()
plt.show()


Let us generate random data based on these distributions, selecting an arbitrary length N. This choice aligns with the objective of the first problem, namely to achieve a low estimation error.

In [147]:
# Number of data
N = 1000   

data_gaussian_1 =np.random.normal(mu_gaussian_1, sigma_gaussian_1, N)
data_gaussian_2 =np.random.normal(mu_gaussian_2, sigma_gaussian_2, N)

data_uniform = np.random.uniform(low=a_uniform, high=b_uniform, size=N)

data_beta = np.random.beta(alpha_beta, beta_beta, N)

# Problem 1: Error Bars 

First, we are going to study the number of data necessaty to have a low error bar. In the following table, you can see the theorical values that we have to use to study this problem

| Distribution | Parameters | Mean ($\mu$) | Variance ($\sigma^2$) |
|-------------|------------|----------------|----------------------|
| Gaussian (Normal) | $\mu, \sigma$ | $\mu$ | $\sigma^2$ |
| Uniform | $a, b$ | $\frac{a+b}{2}$ | $\frac{(b-a)^2}{12}$ |
| Beta | $\alpha, \beta$ | $\frac{\alpha}{\alpha+\beta}$ | $\frac{\alpha \beta}{(\alpha+\beta)^2 (\alpha+\beta+1)}$ |


In the following cell, we determine those theoretical values.

In [148]:
# Teorical means and variances
mean_gaussian_teo_1=mu_gaussian_1
var_gaussian_teo_1=sigma_gaussian_1**2

mean_gaussian_teo_2=mu_gaussian_2
var_gaussian_teo_2=sigma_gaussian_2**2

mean_uniform_teo=(a_uniform+b_uniform)/2.0
var_uniform_teo=(b_uniform-a_uniform)**2/12.0

mean_beta_teo=alpha_beta/(alpha_beta+beta_beta)
var_beta_teo=alpha_beta*beta_beta/((alpha_beta+beta_beta+1.0)*(alpha_beta+beta_beta)**2)

# Create a formatted table
print(f"{'Distribution':<10} | {'Theoretical Mean':>17} | {'Theoretical Var':>17} ")
print("-"*70)
print(f"{'Gaussian 1'} |  {mean_gaussian_teo_1:17.4f} | {var_gaussian_teo_1:17.4f} ")
print(f"{'Gaussian 2'} |  {mean_gaussian_teo_2:17.4f} | {var_gaussian_teo_2:17.4f}")
print(f"{'Uniform'}    |  {mean_uniform_teo:17.4f} | {var_uniform_teo:17.4f} ")
print(f"{'Beta'}       |  {mean_beta_teo:17.4f} | {var_beta_teo:17.4f} ")



Distribution |  Theoretical Mean |   Theoretical Var 
----------------------------------------------------------------------
Gaussian 1 |             2.0000 |            2.2500 
Gaussian 2 |             2.0000 |            0.2500
Uniform    |             0.5000 |            0.0833 
Beta       |             0.3750 |            0.0260 


In order to determinate de error bars for the emperical data, we're going to use two inequalities:

## 1. Hoeffding Inequality

For a set of independent random variables $X_1, X_2, ..., X_N$ bounded in $[a, b]$, the probability that the empirical mean $\bar{X}$ deviates from the expected value $\mathbb{E}[\bar{X}]$ by more than $\epsilon$ is bounded by:

$$
P(|\bar{X} - \mathbb{E}[\bar{X}]| \ge \epsilon) \le 2 \exp\Bigg(-\frac{2 N \epsilon^2}{(b-a)^2}\Bigg)
$$

The error bar (maximum deviation $\epsilon$) with confidence $1-\delta$ is:

$$
\epsilon_{\text{Hoeffding}} \le \sqrt{\frac{(b-a)^2 \ln(2/\delta)}{2 N}}
$$

---

## 2. Chebyshev Inequality

For a random variable with mean $\mu$ and variance $\sigma^2$, the probability that the sample mean $\bar{X}$ deviates from the true mean by more than $\epsilon$ is bounded by:

$$
P(|\bar{X} - \mu| \ge \epsilon) \le \frac{\sigma^2}{N \epsilon^2}
$$

The error bar (maximum deviation $\epsilon$) with confidence $1-\delta$ is:
$$
\epsilon_{\text{Chebyshev}} \le \sqrt{\frac{\sigma^2}{N \delta}}
$$
This inequality is better to be applied when $\sigma$ is small or it's known.

In that way, if we know $\epsilon$  we can say that:
$$
\bar{X}-\epsilon \le \mu \le \bar{X} + \epsilon
$$


In [149]:
def hoeffding_errorbar(data, delta=0.05):

    N = len(data)
    a, b = np.min(data), np.max(data) 
    
    eps = np.sqrt(((b - a)**2 * np.log(2/delta)) / (2 * N))
    return eps

def chebyshev_errorbar(data, var, delta=0.05):
    N = len(data)
    eps = np.sqrt(var / (N * delta))
    return eps


In [150]:
# Hoeffding error bars
eps_hoeff_gaussian_1 = hoeffding_errorbar(data_gaussian_1, delta=0.05)
eps_hoeff_gaussian_2 = hoeffding_errorbar(data_gaussian_2, delta=0.05)
eps_hoeff_uniform = hoeffding_errorbar(data_uniform, delta=0.05)
eps_hoeff_beta = hoeffding_errorbar(data_beta, delta=0.05)

# Chebyshev error bars
eps_cheb_gaussian_1 = chebyshev_errorbar(data_gaussian_1,var_gaussian_teo_1, delta=0.05)
eps_cheb_gaussian_2 = chebyshev_errorbar(data_gaussian_2,var_gaussian_teo_2, delta=0.05)
eps_cheb_uniform = chebyshev_errorbar(data_uniform,var_uniform_teo, delta=0.05)
eps_cheb_beta = chebyshev_errorbar(data_beta,var_beta_teo, delta=0.05)

# Print Hoeffding vs Chebyshev error bars in a table
print(f"{'Distribution':<10} | {'Hoeffding Error':>15} | {'Chebyshev Error':>15}")
print("-"*45)
print(f"{'Gaussian 1'}   | {eps_hoeff_gaussian_1:15.4f} | {eps_cheb_gaussian_1:15.4f}")
print(f"{'Gaussian 2'}   | {eps_hoeff_gaussian_2:15.4f} | {eps_cheb_gaussian_2:15.4f}")
print(f"{'Uniform'}      | {eps_hoeff_uniform:15.4f} | {eps_cheb_uniform:15.4f}")
print(f"{'Beta'}         | {eps_hoeff_beta:15.4f} | {eps_cheb_beta:15.4f}")


Distribution | Hoeffding Error | Chebyshev Error
---------------------------------------------
Gaussian 1   |          0.3852 |          0.2121
Gaussian 2   |          0.1328 |          0.0707
Uniform      |          0.0429 |          0.0408
Beta         |          0.0338 |          0.0228


We observe that $\epsilon$, which is simply the difference between the empirical and theoretical values, can sometimes be quite large. Therefore, we will fix $\epsilon$ and determine the required data length in order to obtain better results. We can see that this error depends on the variance: for the first Gaussian distribution, with variance greater than one, we observe more dispersion.



Solving for $N$ to achieve a confidence level of $1-\delta$ for the hoeffding Inequality:

$$
N \le \frac{(b-a)^2 \ln(2/\delta)}{2 \epsilon^2}
$$


For the Chebyshev Inequality:

$$
N \le \frac{\sigma^2}{\delta \epsilon^2}
$$



In [151]:
def hoeffding_get_N(data, epsilon=0.01, delta=0.05):
    a, b = np.min(data), np.max(data)
    N = ((b - a)**2 * np.log(2/delta)) / (2 * epsilon**2)
    return int(np.ceil(N))

def chebyshev_get_N(var, epsilon=0.01, delta=0.05):
    N = var / (delta * epsilon**2)
    return int(np.ceil(N))

We will choose a small value for $\epsilon$ (0.01) and a confidence level $\delta$ (0.05), and then estimate the required data length $N$.


In [152]:
# Epsilon and delta
epsilon = 0.01
delta = 0.05

# Estimate N using Hoeffding
N_hoeff_gaussian_1 = hoeffding_get_N(data_gaussian_1, epsilon, delta)
N_hoeff_gaussian_2 = hoeffding_get_N(data_gaussian_2, epsilon, delta)
N_hoeff_uniform = hoeffding_get_N(data_uniform, epsilon, delta)
N_hoeff_beta = hoeffding_get_N(data_beta, epsilon, delta)

# Estimate N using Chebyshev
N_cheb_gaussian_1 = chebyshev_get_N(var_gaussian_teo_1, epsilon, delta)
N_cheb_gaussian_2 = chebyshev_get_N(var_gaussian_teo_2, epsilon, delta)
N_cheb_uniform = chebyshev_get_N(var_uniform_teo, epsilon, delta)
N_cheb_beta = chebyshev_get_N(var_beta_teo, epsilon, delta)


print(f"{'Distribution':<10} | {'Hoeffding N':>15} | {'Chebyshev N':>15}")
print("-"*45)
print(f"{'Gaussian 1'}   | {N_hoeff_gaussian_1:15.0f} | {N_cheb_gaussian_1:15.0f}")
print(f"{'Gaussian 2'}   | {N_hoeff_gaussian_2:15.0f} | {N_cheb_gaussian_2:15.0f}")
print(f"{'Uniform'}      | {N_hoeff_uniform:15.0f} | {N_cheb_uniform:15.0f}")
print(f"{'Beta'}         | {N_hoeff_beta:15.0f} | {N_cheb_beta:15.0f}")

Distribution |     Hoeffding N |     Chebyshev N
---------------------------------------------
Gaussian 1   |         1483417 |          450000
Gaussian 2   |          176463 |           50000
Uniform      |           18408 |           16667
Beta         |           11432 |            5209


Now that we have determined the required data length, we can see that the Hoeffding bound is larger than the Chebyshev bound. We can infer that Hoeffding overestimates the number $N$, since it depends on the upper and lower bounds $a$ and $b$, which in our case we set as the maximum and minimum data values. Therefore, we will use the Chebyshev bound and verify whether it holds or not.  Moreover, for each distribution we know the theoretical value of the variance, and as we argued before, it is preferable to use this inequality in this case.

Once we know the amount of data needed to achieve small error bars, we will generate new data and compare the theoretical and empirical values.



# Problem 2 : Theoretical Values of Mean and Variance

To compare the experimental and theoretical values, we compute the relative error:

$$
e_{\text{rel}}(\%) =
\left| 
\frac{ \text{value}_{\text{exp}} - \text{value}_{\text{teo}} }
     { \text{value}_{\text{teo}} }
\right|
\cdot 100
$$

Also, we use $\epsilon=\text{abs}(\mu-\bar{X})$. Based on the previous problem, we can choose a new data length in order to obtain a lower error bar.

In [169]:
# Using the estimated N to generate new datasets

N_data_gaussian_1 = N_cheb_gaussian_1
N_data_gaussian_2 = N_cheb_gaussian_2
N_data_uniform = N_cheb_uniform
N_data_beta = N_cheb_beta

# Generate new datasets with the estimated N

data_gaussian_1 =np.random.normal(mu_gaussian_1, sigma_gaussian_1, N_data_gaussian_1)
data_gaussian_2 =np.random.normal(mu_gaussian_2, sigma_gaussian_2, N_data_gaussian_2)
data_uniform = np.random.uniform(low=a_uniform, high=b_uniform, size=N_data_uniform)
data_beta = np.random.beta(alpha_beta, beta_beta, N_data_beta)

# Empirical means and variances
mean_gaussian_emp_1 = np.mean(data_gaussian_1)
var_gaussian_emp_1 = np.var(data_gaussian_1) 

mean_gaussian_emp_2 = np.mean(data_gaussian_2)
var_gaussian_emp_2 = np.var(data_gaussian_2) 

mean_uniform_emp  = np.mean(data_uniform)
var_uniform_emp  = np.var(data_uniform)

mean_beta_emp     = np.mean(data_beta)
var_beta_emp     = np.var(data_beta)

# Relative error calculation
rel_error_mean_gaussian_1 = abs(mean_gaussian_emp_1 - mean_gaussian_teo_1) / mean_gaussian_teo_1 * 100
rel_error_var_gaussian_1  = abs(var_gaussian_emp_1 - var_gaussian_teo_1) / var_gaussian_teo_1 * 100
epsilon_error_gaussian_1 = abs(mean_gaussian_emp_1 - mean_gaussian_teo_1)

rel_error_mean_gaussian_2 = abs(mean_gaussian_emp_2 - mean_gaussian_teo_2) / mean_gaussian_teo_2 * 100
rel_error_var_gaussian_2  = abs(var_gaussian_emp_2 - var_gaussian_teo_2) / var_gaussian_teo_2 * 100
epsilon_error_gaussian_2 = abs(mean_gaussian_emp_2 - mean_gaussian_teo_2)

rel_error_mean_uniform = abs(mean_uniform_emp - mean_uniform_teo) / mean_uniform_teo * 100
rel_error_var_uniform  = abs(var_uniform_emp - var_uniform_teo) / var_uniform_teo * 100
epsilon_error_uniform = abs(mean_uniform_emp - mean_uniform_teo)

rel_error_mean_beta = abs(mean_beta_emp - mean_beta_teo) / mean_beta_teo * 100
rel_error_var_beta  = abs(var_beta_emp - var_beta_teo) / var_beta_teo * 100
epsilon_error_beta = abs(mean_beta_emp - mean_beta_teo)

# Create a formatted table for mean
print(f"{'Distribution':<10} | {'Empirical Mean':>15} | {'Theoretical Mean':>17} | {'Rel. Error (%)':>15} | {'Epsilon Error':>15}")
print("-"*90)
print(f"{'Gaussian 1'} | {mean_gaussian_emp_1:15.4f} | {mean_gaussian_teo_1:17.4f} | {rel_error_mean_gaussian_1:15.4f} | {epsilon_error_gaussian_1:15.4f}")
print(f"{'Gaussian 2'} | {mean_gaussian_emp_2:15.4f} | {mean_gaussian_teo_2:17.4f} | {rel_error_mean_gaussian_2:15.4f} | {epsilon_error_gaussian_2:15.4f}")
print(f"{'Uniform'}    | {mean_uniform_emp:15.4f} | {mean_uniform_teo:17.4f} | {rel_error_mean_uniform:15.4f} | {epsilon_error_uniform:15.4f}")
print(f"{'Beta'}       | {mean_beta_emp:15.4f} | {mean_beta_teo:17.4f} | {rel_error_mean_beta:15.4f} | {epsilon_error_beta:15.4f}")

print("\n")

# Create a formatted table for variances   
print(f"{'Distribution':<10} | {'Empirical Var':>15} | {'Theoretical Var':>17} | {'Rel. Error (%)':>15} ")
print("-"*70)
print(f"{'Gaussian 1'} | {var_gaussian_emp_1:15.4f} | {var_gaussian_teo_1:17.4f} | {rel_error_var_gaussian_1:15.4f} ")
print(f"{'Gaussian 2'} | {var_gaussian_emp_2:15.4f} | {var_gaussian_teo_2:17.4f} | {rel_error_var_gaussian_2:15.4f} ")
print(f"{'Uniform'}    | {var_uniform_emp:15.4f} | {var_uniform_teo:17.4f} | {rel_error_var_uniform:15.4f} ")
print(f"{'Beta'}       | {var_beta_emp:15.4f} | {var_beta_teo:17.4f} | {rel_error_var_beta:15.4f} ")

Distribution |  Empirical Mean |  Theoretical Mean |  Rel. Error (%) |   Epsilon Error
------------------------------------------------------------------------------------------
Gaussian 1 |          1.9980 |            2.0000 |          0.0981 |          0.0020
Gaussian 2 |          1.9972 |            2.0000 |          0.1418 |          0.0028
Uniform    |          0.4994 |            0.5000 |          0.1219 |          0.0006
Beta       |          0.3752 |            0.3750 |          0.0500 |          0.0002


Distribution |   Empirical Var |   Theoretical Var |  Rel. Error (%) 
----------------------------------------------------------------------
Gaussian 1 |          2.2537 |            2.2500 |          0.1624 
Gaussian 2 |          0.2505 |            0.2500 |          0.2175 
Uniform    |          0.0834 |            0.0833 |          0.0718 
Beta       |          0.0258 |            0.0260 |          0.8554 


We can see that the relative error of the mean is less than or equal to one, and the $\epsilon$ error (first table, last column) is below 0.01, which is the value we selected. By choosing the data length according to the Chebyshev bound, we can see that it is sufficient. Moreover, the empirical and theoretical variances are quite similar.




## Problem 3: Checking the 68–95–99.7 Rule

In this part, we investigate which distributions satisfy the **68–95–99.7 rule**.

We study the fraction of the data that lies within the following intervals:

- $\mu \pm \sigma$
- $\mu \pm 2\sigma$
- $\mu \pm 3\sigma$

and compare it with the theoretical percentages:

- 68% of the data within $\mu \pm \sigma$
- 95% of the data within $\mu \pm 2\sigma$
- 99.7% of the data within $\mu \pm 3\sigma$


In [170]:
distributions = {
    "Gaussian 1": data_gaussian_1,
    "Gaussian 2": data_gaussian_2,
    "Uniform": data_uniform,
    "Beta": data_beta
}

print(f"{'Distribution':<10} | {'1σ (%)':<8} | {'2σ (%)':<8} | {'3σ (%)':<8}")
print("-"*40)

for name, data in distributions.items():
    mu = np.mean(data)
    sigma = np.std(data)
    
    frac_1sigma = np.mean((data >= mu - sigma) & (data <= mu + sigma)) * 100
    frac_2sigma = np.mean((data >= mu - 2*sigma) & (data <= mu + 2*sigma)) * 100
    frac_3sigma = np.mean((data >= mu - 3*sigma) & (data <= mu + 3*sigma)) * 100
    
    print(f"{name:<10} | {frac_1sigma:>7.2f} | {frac_2sigma:>7.2f} | {frac_3sigma:>7.2f}")


Distribution | 1σ (%)   | 2σ (%)   | 3σ (%)  
----------------------------------------
Gaussian 1 |   68.37 |   95.44 |   99.72
Gaussian 2 |   68.26 |   95.37 |   99.76
Uniform    |   57.55 |  100.00 |  100.00
Beta       |   66.44 |   96.35 |   99.88


From the results, we can see that only the Gaussian (normal) distribution approximately satisfies the 68–95–99.7 rule. The Uniform and Beta distributions do not follow this empirical rule. The case of the Beta distribution may satisfy this rule only if its shape is sufficiently similar to a Gaussian.